In [1]:
import pandas as pd
pd.set_option("display.max_rows",None)
from time import sleep

from nba_api.stats.static import players
from nba_api.stats.endpoints import playerawards, playercareerstats
from selenium import webdriver
from selenium.webdriver.common.by import By
from io import StringIO

In [22]:
#same setup process as the inactives, just with get_active players
actives=pd.DataFrame(players.get_active_players())
actives

,id,full_name,first_name,last_name,is_active
0,1630173,Precious Achiuwa,Precious,Achiuwa,True
1,203500,Steven Adams,Steven,Adams,True
2,1628389,Bam Adebayo,Bam,Adebayo,True
3,1630534,Ochai Agbaji,Ochai,Agbaji,True
4,1630583,Santi Aldama,Santi,Aldama,True
5,1641725,Trey Alexander,Trey,Alexander,True
6,1629638,Nickeil Alexander-Walker,Nickeil,Alexander-Walker,True
7,1628960,Grayson Allen,Grayson,Allen,True
8,1628386,Jarrett Allen,Jarrett,Allen,True
9,1630631,Jose Alvarado,Jose,Alvarado,True


In [3]:
test_df=actives.iloc[:5].copy()
test_df

,id,full_name,first_name,last_name,is_active
0,1630173,Precious Achiuwa,Precious,Achiuwa,True
1,203500,Steven Adams,Steven,Adams,True
2,1628389,Bam Adebayo,Bam,Adebayo,True
3,1630534,Ochai Agbaji,Ochai,Agbaji,True
4,1630583,Santi Aldama,Santi,Aldama,True


In [4]:
#same as from inactives, might need to verify column order is still correct
def rename_avgs(col):
    if col=="PF":
        rename="F"
    elif col=="OREB" or col =="DREB":
        rename=col[:2]
    elif col[0]=="F":
        rename=col
    else:
        rename=col[0]
    return rename+"PG"
def clean_avgs(df):
    df.rename(columns=rename_avgs,inplace=True)
    return pd.concat([df.iloc[0,5:8],df.iloc[0,9:11],df.iloc[0,12:14],df.iloc[0,15:]])

In [ ]:
#exact same as inactive player function, minus the need to check for II
def get_career_stats(row):
    # sleep to respect rate limiting
    sleep(0.5)
    #holds the various dfs-season, career, reg season, playoffs
    totals=playercareerstats.PlayerCareerStats(row["id"]).get_data_frames()
    #no need to check for inactive ineligible since they're all active
    sleep(0.5)
    avgs=playercareerstats.PlayerCareerStats(row["id"],per_mode36="PerGame").get_data_frames()
    #check if player hasn't played in the playoffs; if so, only return regular season
    if len(totals[3])==0:
        return pd.concat([totals[1].iloc[0,3:],clean_avgs(avgs[1])])
    return pd.concat([totals[1].iloc[0,3:],clean_avgs(avgs[1]),totals[3].iloc[0,3:].add_prefix("PF_"),clean_avgs(avgs[3]).add_prefix("PF_")])

In [6]:
mosey=actives[actives["full_name"]=="Ryan Rollins"]
mosey

,id,full_name,first_name,last_name,is_active
400,1631157,Ryan Rollins,Ryan,Rollins,True


In [12]:
get_career_stats(mosey)

GP                 116
GS                  49
MIN               2065
FGM                385
FGA                798
FG_PCT        0.482456
FG3M               141
FG3A               337
FG3_PCT       0.418398
FTM                113
FTA                144
FT_PCT        0.784722
OREB                52
DREB               229
REB                281
AST                328
STL                109
BLK                 34
TOV                161
PF                 205
PTS               1024
MPG               17.8
FGMPG              3.3
FGAPG              6.9
FG3MPG             1.2
FG3APG             2.9
FTMPG              1.0
FTAPG              1.2
ORPG               0.4
DRPG               2.0
RPG                2.4
APG                2.8
SPG                0.9
BPG                0.3
TPG                1.4
FPG                1.8
PPG                8.8
PF_GP                3
PF_GS                1
PF_MIN              29
PF_FGM               4
PF_FGA              13
PF_FG_PCT     0.307692
PF_FG3M    

In [13]:
def get_awards(row):
    #get list of all player's awards
    awards=playerawards.PlayerAwards(row["id"]).get_data_frames()[0]
    #used to distinguish 1st, 2nd, 3rd team All-NBA, etc
    team_nums={"1":"1st","2":"2nd","3":"3rd"}
    #get the subset of all the awards with a numbered team and prepend the description with the correct prefix from the dict
    awards.loc[awards["ALL_NBA_TEAM_NUMBER"].fillna("").str.isnumeric(),"DESCRIPTION"]=awards["ALL_NBA_TEAM_NUMBER"].map(team_nums)+" Team "+awards["DESCRIPTION"]
    #count number of times player has won each award and return that; no need to determine HOF for active players
    return awards.groupby("DESCRIPTION").size()

In [14]:
giannis=actives[actives["full_name"]=="Giannis Antetokounmpo"]
giannis

,id,full_name,first_name,last_name,is_active
12,203507,Giannis Antetokounmpo,Giannis,Antetokounmpo,True


In [15]:
get_awards(giannis)

DESCRIPTION
1st Team All-Defensive Team              4
1st Team All-NBA                         7
1st Team NBA Cup All-Tournament Team     2
2nd Team All-Defensive Team              1
2nd Team All-NBA                         2
2nd Team All-Rookie Team                 1
NBA All-Star                             9
NBA All-Star Most Valuable Player        1
NBA Champion                             1
NBA Cup Most Valuable Player             1
NBA Defensive Player of the Year         1
NBA Finals Most Valuable Player          1
NBA Most Improved Player                 1
NBA Most Valuable Player                 2
NBA Player of the Month                  9
NBA Player of the Week                  20
dtype: int64

In [16]:
pd.concat([test_df,test_df.apply(get_career_stats,axis=1),test_df.apply(get_awards,axis=1)],axis=1).fillna(0)

,id,full_name,first_name,last_name,is_active,APG,AST,BLK,BPG,DREB,DRPG,FG3A,FG3APG,FG3M,FG3MPG,FG3_PCT,FGA,FGAPG,FGM,FGMPG,FG_PCT,FPG,FTA,FTAPG,FTM,FTMPG,FT_PCT,GP,GS,MIN,MPG,OREB,ORPG,PF,PF_APG,PF_AST,PF_BLK,PF_BPG,PF_DREB,PF_DRPG,PF_FG3A,PF_FG3APG,PF_FG3M,PF_FG3MPG,PF_FG3_PCT,PF_FGA,PF_FGAPG,PF_FGM,PF_FGMPG,PF_FG_PCT,PF_FPG,PF_FTA,PF_FTAPG,PF_FTM,PF_FTMPG,PF_FT_PCT,PF_GP,PF_GS,PF_MIN,PF_MPG,PF_OREB,PF_ORPG,PF_PF,PF_PPG,PF_PTS,PF_REB,PF_RPG,PF_SPG,PF_STL,PF_TOV,PF_TPG,PPG,PTS,REB,RPG,SPG,STL,TOV,TPG,1st Team All-Defensive Team,2nd Team All-Defensive Team,2nd Team All-Rookie Team,NBA All-Star,NBA Player of the Week,Olympic Gold Medal
0,1630173,Precious Achiuwa,Precious,Achiuwa,True,1.0,347,221,0.6,1298,3.7,428,1.2,130,0.4,0.303738,2204,6.3,1067,3.1,0.484120,1.8,601,1.7,358,1.0,0.595674,348,87,6952,20.0,663,1.9,617,0.4,11.0,19.0,0.7,49.0,1.9,20.0,0.8,5.0,0.2,0.250000,113.0,4.3,55.0,2.1,0.486726,1.4,31.0,1.2,14.0,0.5,0.451613,26.0,3.0,397.0,15.3,33.0,1.3,37.0,5.0,129.0,82.0,3.2,0.2,6.0,18.0,0.7,7.5,2622,1961,5.6,0.6,203,328,0.9,0.0,0.0,0.0,0.0,0.0,0.0
1,203500,Steven Adams,Steven,Adams,True,1.5,1180,717,0.9,3452,4.4,17,0.0,1,0.0,0.058824,4896,6.2,2858,3.6,0.583742,2.3,2192,2.8,1175,1.5,0.536040,789,651,20280,25.7,2874,3.6,1852,0.9,68.0,67.0,0.9,328.0,4.5,3.0,0.0,0.0,0.0,0.000000,365.0,5.0,224.0,3.1,0.613699,2.4,189.0,2.6,101.0,1.4,0.534392,73.0,46.0,1879.0,25.7,204.0,2.8,174.0,7.5,549.0,532.0,7.3,0.5,33.0,69.0,0.9,8.7,6892,6326,8.0,0.8,663,1110,1.4,0.0,0.0,1.0,0.0,0.0,0.0
2,1628389,Bam Adebayo,Bam,Adebayo,True,3.6,2116,513,0.9,3979,6.7,440,0.7,139,0.2,0.315909,6775,11.4,3604,6.1,0.531956,2.4,2721,4.6,2059,3.5,0.756707,595,491,18223,30.6,1312,2.2,1417,3.5,270.0,49.0,0.6,518.0,6.6,37.0,0.5,9.0,0.1,0.243243,975.0,12.5,504.0,6.5,0.516923,2.9,345.0,4.4,261.0,3.3,0.756522,78.0,73.0,2709.0,34.7,197.0,2.5,227.0,16.4,1278.0,715.0,9.2,0.9,67.0,180.0,2.3,15.8,9406,5291,8.9,1.1,635,1271,2.1,1.0,4.0,0.0,3.0,2.0,2.0
3,1630534,Ochai Agbaji,Ochai,Agbaji,True,1.2,271,95,0.4,446,1.9,732,3.2,251,1.1,0.342896,1471,6.4,659,2.9,0.447995,1.7,194,0.8,145,0.6,0.747423,229,106,5045,22.0,198,0.9,396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.5,1714,644,2.8,0.6,135,167,0.7,0.0,0.0,0.0,0.0,0.0,0.0
4,1630583,Santi Aldama,Santi,Aldama,True,2.0,545,165,0.6,1119,4.2,1112,4.1,385,1.4,0.346223,2261,8.4,1045,3.9,0.462185,1.5,442,1.6,305,1.1,0.690045,269,79,6271,23.3,334,1.2,395,1.4,14.0,1.0,0.1,41.0,4.1,39.0,3.9,17.0,1.7,0.435897,77.0,7.7,36.0,3.6,0.467532,1.2,2.0,0.2,2.0,0.2,1.000000,10.0,1.0,222.0,22.2,9.0,0.9,12.0,9.1,91.0,50.0,5.0,0.3,3.0,7.0,0.7,10.3,2780,1453,5.4,0.7,179,261,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
driver=webdriver.Firefox()
driver.get("https://www.basketball-reference.com/players/a/allenti01.html")
avgs=driver.find_element(By.ID,"per_game_stats")
avgs_df=pd.read_html(StringIO(avgs.get_attribute("outerHTML")))[0]
totals=driver.find_element(By.ID,"totals_stats")
totals_df=pd.read_html(StringIO(totals.get_attribute("outerHTML")))[0]
driver.quit()


In [20]:

pd.concat([avgs_df.iloc[1,7:10],avgs_df.iloc[1,11:13],avgs_df.iloc[1,18:20],avgs_df.iloc[1,21:-1]])

MP     25.0
FG      1.2
FGA     4.6
3P      0.0
3PA     1.4
FT      0.2
FTA     0.4
ORB     0.8
DRB     2.6
TRB     3.4
AST     1.0
STL     0.8
BLK     0.0
TOV     0.4
PF      3.6
PTS     2.6
Name: 1, dtype: object

In [21]:
pd.concat([totals_df.iloc[1,5:14],totals_df.iloc[1,18:-2]])

G          5
GS         0
MP       125
FG         6
FGA       23
FG%    0.261
3P         0
3PA        7
3P%      0.0
FT         1
FTA        2
FT%      0.5
ORB        4
DRB       13
TRB       17
AST        5
STL        4
BLK        0
TOV        2
PF        18
PTS       13
Name: 1, dtype: object